In [ ]:
from vpython import *
import numpy as np
import time
from random import random

# Custom Colors using vectors (RGB values between 0 and 1)
darkblue = vector(0, 0, 0.2)
gold = vector(1, 0.843, 0)  # RGB for gold
cyan = vector(0.5, 0.9, 0)
purple = vector(0.5, 0, 0.5)
green = vector(0, 1, 0)
orange = vector(1, 0.647, 0)
red = vector(1, 0, 0)
blue = vector(0, 0, 1)
white = vector(1, 1, 1)

# Set up 3D scene with a custom background color
scene = canvas(width=1200, height=800, background=darkblue)

# Add lighting with a variety of colors for better ambiance
scene.lights = []
distant_light(direction=vector(1, 1, 1), color=white)
distant_light(direction=vector(-1, 1, -1), color=orange)
distant_light(direction=vector(0, -1, 1), color=purple)

# Add the cylindrical enclosure (beam pipe) with a shiny metallic finish
enclosure_radius = 10
enclosure_length = 30
enclosure = cylinder(pos=vector(-enclosure_length / 2, 0, 0), axis=vector(enclosure_length, 0, 0),
                     radius=enclosure_radius, color=vector(0.4, 0.4, 0.4), opacity=0.3, texture=textures.rough)

# Add particle beam effect with a glowing effect
particle_beam = cylinder(pos=vector(-enclosure_length / 2, 0, 0), axis=vector(enclosure_length, 0, 0),
                        radius=0.1, color=orange, opacity=0.3)
glow = cylinder(pos=vector(-enclosure_length / 2, 0, 0), axis=vector(enclosure_length, 0, 0),
                radius=0.2, color=cyan, opacity=0.05)

# Add support structures with metallic accents
for z in np.linspace(-enclosure_length / 2, enclosure_length / 2, 10):
    support = box(pos=vector(z, -enclosure_radius - 1, 0), size=vector(0.5, 2, 0.5), color=vector(0.75, 0.75, 0.75))

# Add control room details with metallic texture and bright windows
control_room = box(pos=vector(0, -enclosure_radius - 5, 0), size=vector(10, 5, 10), color=vector(0.3, 0.1, 0.2))

for x in np.linspace(-4, 4, 3):
    window = box(pos=vector(x, -enclosure_radius - 2.5, 5), size=vector(2, 2, 0.1), color=green, opacity=0.7)
    
light1 = sphere(pos=vector(-4, -enclosure_radius - 3, 5), radius=0.2, color=red)
light2 = sphere(pos=vector(4, -enclosure_radius - 3, 5), radius=0.2, color=color.yellow)

# Add circular bending magnets with gold and silver texture for diversity
num_magnets = 12
for theta in np.linspace(0, 2 * np.pi, num_magnets):
    magnet_pos = vector(enclosure_radius * np.cos(theta), enclosure_radius * np.sin(theta), 0)
    ring(pos=magnet_pos, axis=vector(0, 0, 1), radius=0.5, thickness=0.1, color=gold, texture=textures.metal)

# Add detector rings with a shiny, futuristic look
detector1 = ring(pos=vector(0, 0, 0), axis=vector(0, 0, 1), radius=12, thickness=0.5, color=cyan)

# Add cryogenic cooling pipes with a shimmering effect (e.g., green pipes for cooling)
for z in np.linspace(-enclosure_length / 2, enclosure_length / 2, 20):
    cooling_pipe = cylinder(pos=vector(z, enclosure_radius + 1, 0), axis=vector(0, 0.5, 0), radius=0.2, color=green)
    insulation = cylinder(pos=vector(z, enclosure_radius + 1, 0), axis=vector(0, 0.5, 0), radius=0.25, color=vector(0.6, 0.6, 0.6), opacity=0.6)

# Add cooling tanks with blue and purple gradient colors for a more vibrant look
cooling_tank1 = cylinder(pos=vector(-enclosure_length / 2, enclosure_radius + 2, 0), axis=vector(0, 1, 0), radius=1, color=blue)
cooling_tank2 = cylinder(pos=vector(enclosure_length / 2, enclosure_radius + 2, 0), axis=vector(0, 1, 0), radius=1, color=purple)

# Create a smaller legend in the top-left corner with a background box
legend_offset = vector(-26, 5, 0)
legend_data = [("Proton", color.red), ("Neutron", color.cyan), ("π⁺ Pion", vector(1, 0, 1))]
legend_spacing = 1.5  # Increased spacing to avoid overlap
radius = 0.3
text_offset = vector(2.5, 0, 0)  # Adjusted so labels don't overlap with spheres

# Background box for the legend
legend_width = 5
legend_height = legend_spacing * len(legend_data) + 0.5
legend_bg = box(pos=legend_offset + vector(1, -legend_spacing, -0.1), 
                size=vector(legend_width, legend_height, 0.2), 
                color=darkblue, opacity=0.5)

# Create legend entries
for i, (name, col) in enumerate(legend_data):
    y_offset = -legend_spacing * i
    sphere(pos=legend_offset + vector(-0.5, y_offset, 0), radius=radius, color=col)
    label(pos=legend_offset + vector(2, y_offset, 0), text=name, height=12, color=color.white, box=False)

# Define magnetic field (simulated Lorentz force remains unchanged)
B_field = vector(0, 0, 0.2)

# Add a title
title = label(pos=vector(0, 20, 0), text="Particle Collision Simulator", height=30, color=color.white, box=False)

# Particle class remains the same, with particle colors already specified
class Particle:
    def __init__(self, pos, velocity, radius, mass, charge, particle_type, label):
        self.pos = pos
        self.velocity = velocity
        self.radius = radius
        self.mass = mass
        self.charge = charge
        self.particle_type = particle_type
        self.label = label
        self.energy = 0.5 * self.mass * mag(self.velocity) ** 2
        
        if self.particle_type == 'proton':
            self.particle_color = color.red
        elif self.particle_type == 'neutron':
            self.particle_color = color.cyan
        elif self.particle_type == 'π⁺':
            self.particle_color = vector(1, 0, 1)  # Magenta
        
        # Modify the Particle class to include glowing trails
        self.sphere = sphere(pos=self.pos, radius=self.radius, color=self.particle_color, 
                             make_trail=True, trail_type="curve", interval=5, retain=50, 
                             trail_color=self.particle_color, trail_radius=0.02, trail_opacity=0.5)

    def update(self, dt):
        force = self.charge * cross(self.velocity, B_field)  # Lorentz force
        acceleration = force / self.mass
        self.velocity += acceleration * dt
        self.pos += self.velocity * dt
        self.sphere.pos = self.pos
        self.energy = 0.5 * self.mass * mag(self.velocity) ** 2


# Reduce initial velocities
particle1 = Particle(pos=vector(-5, 0, 0), velocity=vector(1, 0.5, 0), radius=0.5, mass=1, charge=1, particle_type='proton', label='p1')
particle2 = Particle(pos=vector(5, 0, 0), velocity=vector(-1, -0.5, 0), radius=0.5, mass=1, charge=1, particle_type='proton', label='p2')

# Update particle list
particles = [particle1, particle2]

collision_counter = 0

# Update particle counts
particle_counts = {'proton': 2, 'neutron': 0, 'π⁺': 0}

# Function to check for collisions
def are_colliding(p1, p2):
    return mag(p1.pos - p2.pos) <= (p1.radius + p2.radius)

# Function to handle collisions
sparks = []
def handle_collision(p1, p2):
    global collision_counter
    collision_counter += 1

    # Generate explosion effect
    for _ in range(20):  # Number of sparks
        spark_pos = p1.pos + vector(random() * 2 - 1, random() * 2 - 1, random() * 2 - 1)
        spark = sphere(pos=spark_pos, radius=0.1, color=color.yellow, opacity=0.8)
        spark.velocity = vector(random() * 4 - 2, random() * 4 - 2, random() * 4 - 2)
        sparks.append(spark)

    # Shockwave effect
    shockwave = ring(pos=p1.pos, axis=vector(0, 1, 0), radius=1, thickness=0.1, color=color.white, opacity=0.5)
    shockwave.expansion_rate = 5  # Expansion speed
    sparks.append(shockwave)

    # Remove collided particles
    p1.sphere.visible = False
    p2.sphere.visible = False
    particles.remove(p1)
    particles.remove(p2)

    # Create new particles (1 proton, 1 neutron, 1 pion)
    proton = Particle(
        pos=p1.pos + vector(1, 0, 0),
        velocity=vector(random() * 4 - 2, random() * 4 - 2, random() * 4 - 2),
        radius=0.5,
        mass=1,
        charge=1,
        particle_type='proton',
        label=f'p{collision_counter}'
    )
    particles.append(proton)

    neutron = Particle(
        pos=p1.pos + vector(0, 1, 0),
        velocity=vector(random() * 4 - 2, random() * 4 - 2, random() * 4 - 2),
        radius=0.5,
        mass=1,
        charge=0,
        particle_type='neutron',
        label=f'n{collision_counter}'
    )
    particles.append(neutron)

    pion = Particle(
        pos=p1.pos + vector(-1, 0, 0),
        velocity=vector(random() * 4 - 2, random() * 4 - 2, random() * 4 - 2),
        radius=0.3,
        mass=0.14,
        charge=1,
        particle_type='π⁺',
        label=f'π⁺{collision_counter}'
    )
    particles.append(pion)

    # Update particle counts
    particle_counts['proton'] += 1
    particle_counts['neutron'] += 1
    particle_counts['π⁺'] += 1

# HUD (Heads-Up Display)
hud = label(pos=vector(-25, 15, 0), text="Particles: 2\nCollisions: 0\nEnergy: 0 GeV", height=20, color=color.white, box=False)

# Simulation loop
dt = 0.05
start_time = time.time()
max_time = 60
# Define maximum collisions
max_collisions = 80  # Stop the simulation after 300 collisions

while time.time() - start_time < max_time:
    rate(90)  # 60 FPS
    
    # Stop simulation if collision counter exceeds max_collisions
    if collision_counter >= max_collisions:
        break
        
    for p in particles:
        p.update(dt)

        # Boundary conditions (particles bounce off the enclosure walls)
        # Improved boundary conditions
        if mag(p.pos) + p.radius >= enclosure_radius:
            normal = p.pos / mag(p.pos)
            p.velocity = p.velocity - 2 * dot(p.velocity, normal) * normal
            p.pos = p.pos - (mag(p.pos) + p.radius - enclosure_radius) * normal  # Move particle back inside
        if mag(p.velocity) > 3:  # Limit max speed to prevent wild spirals
            p.velocity = 3 * norm(p.velocity)

    # Check for collisions between all particle types
    for i in range(len(particles)):
        for j in range(i + 1, len(particles)):
            if are_colliding(particles[i], particles[j]):
                # Handle collisions involving protons or pions
                if particles[i].particle_type in ['proton', 'π⁺'] or particles[j].particle_type in ['proton', 'π⁺']:
                    handle_collision(particles[i], particles[j])

    # Update sparks
    for spark in sparks:
        if hasattr(spark, "velocity"):  # Only move sparks (not shockwaves)
            spark.pos += spark.velocity * dt
        else:  # Shockwave expands instead of moving
            spark.radius += spark.expansion_rate * dt  
    
        spark.opacity -= 0.02
        if spark.opacity <= 0:
            spark.visible = False
            sparks.remove(spark)

    # Update HUD
    total_energy = sum(p.energy for p in particles)
    hud.text = f"Particles: {len(particles)}\nCollisions: {collision_counter}\nEnergy: {total_energy:.2f} GeV\n" \
               f"Protons: {particle_counts['proton']}\nNeutrons: {particle_counts['neutron']}\nPions: {particle_counts['π⁺']}"

# Print simulation completion message
if collision_counter >= max_collisions:
    print(f"Simulation stopped after reaching {max_collisions} collisions.")
else:
    print("Simulation completed.")


<IPython.core.display.Javascript object>